# Inversion Recovery

## Fitting Signal

Lorem ipsum. Lorem ipsum. Lorem ipsum. Lorem ipsum. Lorem ipsum. Lorem ipsum. Lorem ipsum. Lorem ipsum. Lorem ipsum. Lorem ipsum. Lorem ipsum. Lorem ipsum. Lorem ipsum. Lorem ipsum. Lorem ipsum. Lorem ipsum. Lorem ipsum. Lorem ipsum. Lorem ipsum. Lorem ipsum. Lorem ipsum. Lorem ipsum. Lorem ipsum. Lorem ipsum. Lorem ipsum. Lorem ipsum. Lorem ipsum. Lorem ipsum. Lorem ipsum. Lorem ipsum. Lorem ipsum. Lorem ipsum. Lorem ipsum. Lorem ipsum. Lorem ipsum. Lorem ipsum. Lorem ipsum. Lorem ipsum. Lorem ipsum. Lorem ipsum. Lorem ipsum. Lorem ipsum. 


In [1]:
cd ../qMRLab
startup


    startup at line 1 column 1
    startup at line 1 column 1
loading struct
loading io
loading statistics
loading optim
loading image


In [2]:
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
config={'showLink': False}

In [3]:
%% Simulate the signal
%

params.EXC_FA = 90;
params.INV_FA = 180;
params.TI = 50:50:1500;
params.T1 = 1000;
params.TR = 5000;

Mz_analytical = inversion_recovery.analytical_solution(params, 'GRE-IR', 1);


## Fit using non-linear least-squares

In [4]:
TI = params.TI;
y = abs(Mz_analytical);


x0(1) = 0.5;
x0(2) = 900;

fun = @(x) abs(x(1) .* (1 - 2.*exp(-TI./x(2)))) - y;

options.Algorithm = 'levenberg-marquardt';
[x, resnorm] = lsqnonlin(fun,x0,[],[],options);
disp(x(1))
disp(x(2))
disp(resnorm)

 0.99596
 989.06
   2.3123e-05


## Fit using Barral algorithm

In [5]:

irObj = inversion_recovery();
irObj.Prot.IRData.Mat = params.TI';

data.IRData = Mz_analytical;

fitOutput=irObj.fit(data);

disp(fitOutput)


  scalar structure containing the fields:

    T1 =  1000
    rb = -2.0000
    ra =  1.0067
    res =    1.7418e-15
    idx =  13


# Wide range of TRs

In [6]:
params.EXC_FA = 90;
params.INV_FA = 180;
params.TI = 50:50:1500;
params.T1 = 1000;
TR = 1500:50:5000;

for ii = 1:length(TR)
    params.TR = TR(ii);
    Mz_analytical(ii,:) = inversion_recovery.analytical_solution(params, 'GRE-IR', 1);
end


In [7]:
TI = params.TI;


x0(1) = 0.5;
x0(2) = 900;
options.Algorithm = 'levenberg-marquardt';

for ii=1:length(TR)
    y = abs(Mz_analytical(ii,:));
    fun = @(x) abs(x(1) .* (1 - 2.*exp(-TI./x(2)))) - y;

    [fitOutput_nlsq(ii,:), resnorm] = lsqnonlin(fun,x0,[],[],options);
    T1_nlsq(ii) = fitOutput_nlsq(ii,2);
end


In [8]:

irObj = inversion_recovery();
irObj.Prot.IRData.Mat = params.TI';

for ii=1:length(TR)

    data.IRData = Mz_analytical(ii,:);

    fitOutput_barral{ii} = irObj.fit(data);

    T1_barral(ii) = fitOutput_barral{ii}.T1;

end


In [9]:
%get params --from Octave
%get Mz_analytical --from Octave
%get fitOutput_nlsq --from Octave
%get fitOutput_barral --from Octave
%get TR --from Octave

In [19]:
init_notebook_mode(connected=True)

data1 = [dict(
        visible = False,
        mode = 'markers',
        x = params["TI"],
        y = abs(np.squeeze(np.asarray(Mz_analytical[ii]))),
        name = 'Simulated data') for ii in range(len(TR))]

data1[10]['visible'] = True

data2 = [dict(
        visible = False,
        mode = 'lines',
        x = params["TI"],
        y = abs(fitOutput_nlsq[ii,0] * (1 - 2*np.exp(-params['TI']/fitOutput_nlsq[ii,1]))),
        name = 'Fitted T1 [C(1-2*exp(-TI/T1))]:' + str(round(fitOutput_nlsq[ii,1])) + ' ms') for ii in range(len(TR))]

data2[10]['visible'] = True

data3 = [dict(
        visible = False,
        mode = 'lines',
        x = params["TI"],
        y = abs((fitOutput_barral[ii]['ra']+fitOutput_barral[ii]['rb']*np.exp(-params['TI']/fitOutput_barral[ii]['T1']))),
        name = 'Fitted T1 [ra+rb*exp(-TI/T1)]:' + str(round(fitOutput_barral[ii]['T1'])) + ' ms') for ii in range(len(TR))]

data3[10]['visible'] = True



data = data1 + data2 + data3

steps = []
for i in range(len(TR)):
    step = dict(
        method = 'restyle',  
        args = ['visible', [False] * len(data1)],
        label = str(TR[i])
        )
    step['args'][1][i] = True # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    x = 0,
    y = -0.2,
    active = 10,
    currentvalue = {"prefix": "TR value (ms): "},
    pad = {"t": 50},
    steps = steps
)]

layout = go.Layout(
    title='T1 = 1000 ms',
    xaxis=dict(
        autorange=False,
        range=[0, params['TI'][-1]],
        title='Inversion time – TI (ms)',
        titlefont=dict(
            family='Times New Roman',
            size=26
        ),
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    yaxis=dict(
        autorange=False,
        range=[0, 1],
        title='Signal (magnitude)',
        titlefont=dict(
            family='Times New Roman',
            size=26
        ),
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    legend=dict(
        traceorder='normal',
        font=dict(
            family='Times New Roman',
            size=12,
            color='#000'
        ),
        bordercolor='#000000',
        borderwidth=2
    ), 
    sliders=sliders
)

fig = dict(data=data, layout=layout)

iplot(fig, filename = 'basic-line', config = config)